<a href="https://colab.research.google.com/github/StarkArk/home_sales_in_sparksql/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,068 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,735 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main 

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [4]:
# Look at the home sales data
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# Print the schema of the DataFrame
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [11]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')



In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT bedrooms, ROUND(AVG(price),2) FROM home_sales WHERE bedrooms = '4' GROUP BY 1""").show()

+--------+--------------------+
|bedrooms|round(avg(price), 2)|
+--------+--------------------+
|       4|           299661.01|
+--------+--------------------+



In [22]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built AS year_built, bedrooms, bathrooms, ROUND(AVG(price),2) as average_price FROM home_sales WHERE bedrooms='3' AND bathrooms='3' GROUP BY 1, 2, 3 ORDER BY year_built ASC""").show()


+----------+--------+---------+-------------+
|year_built|bedrooms|bathrooms|average_price|
+----------+--------+---------+-------------+
|      2010|       3|        3|    292859.62|
|      2011|       3|        3|    291117.47|
|      2012|       3|        3|    293683.19|
|      2013|       3|        3|    295962.27|
|      2014|       3|        3|    290852.27|
|      2015|       3|        3|     288770.3|
|      2016|       3|        3|    290555.07|
|      2017|       3|        3|    292676.79|
+----------+--------+---------+-------------+



In [25]:
# 4(B). What is the average price of a home for each year homes were sold that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT EXTRACT(YEAR FROM date) AS year_sold, bedrooms, bathrooms, ROUND(AVG(price),2) as average_price
             FROM home_sales WHERE bedrooms='3' AND bathrooms='3' GROUP BY 1, 2, 3 ORDER BY year_sold ASC""").show()

+---------+--------+---------+-------------+
|year_sold|bedrooms|bathrooms|average_price|
+---------+--------+---------+-------------+
|     2019|       3|        3|    287287.82|
|     2020|       3|        3|    294204.16|
|     2021|       3|        3|    294211.46|
|     2022|       3|        3|    292725.69|
+---------+--------+---------+-------------+



In [27]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT
date_built AS year_built,
bedrooms,
bathrooms,
floors,
ROUND(AVG(price),2) as average_price
FROM home_sales
WHERE bedrooms='3' AND bathrooms='3' AND floors='2' AND sqft_living >= 2000
GROUP BY 1, 2, 3, 4
ORDER BY year_built ASC""").show()


+----------+--------+---------+------+-------------+
|year_built|bedrooms|bathrooms|floors|average_price|
+----------+--------+---------+------+-------------+
|      2010|       3|        3|     2|    285010.22|
|      2011|       3|        3|     2|    276553.81|
|      2012|       3|        3|     2|    307539.97|
|      2013|       3|        3|     2|    303676.79|
|      2014|       3|        3|     2|    298264.72|
|      2015|       3|        3|     2|    297609.97|
|      2016|       3|        3|     2|     293965.1|
|      2017|       3|        3|     2|    280317.58|
+----------+--------+---------+------+-------------+



In [46]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
WITH pricey_views AS
  (SELECT view as view_rating, ROUND(AVG(price),2) as average_price
   FROM home_sales
   GROUP BY view_rating
   ORDER BY view_rating DESC)

SELECT CAST(view_rating AS INTEGER) as view_rating, CAST(average_price AS INTEGER) AS avg_price
FROM pricey_views
WHERE average_price >= 350000
ORDER BY view_rating DESC
"""
).show(60)

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|        100|  1026669|
|         99|  1061201|
|         98|  1053739|
|         97|  1129040|
|         96|  1017815|
|         95|  1054325|
|         94|  1033536|
|         93|  1026006|
|         92|   970402|
|         91|  1137372|
|         90|  1062654|
|         89|  1107839|
|         88|  1031719|
|         87|  1072285|
|         86|  1070444|
|         85|  1056336|
|         84|  1117233|
|         83|  1033965|
|         82|  1063498|
|         81|  1053472|
|         80|   991767|
|         79|  1009565|
|         78|  1080649|
|         77|  1076205|
|         76|  1058802|
|         75|  1114042|
|         74|   745077|
|         73|   752861|
|         72|   780914|
|         71|   775651|
|         70|   695865|
|         69|   750537|
|         68|   716785|
|         67|   737970|
|         66|   712475|
|         65|   736679|
|         64|   767036|
|         63|   711614|
|         62|   

In [47]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [48]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [49]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
WITH pricey_views AS
  (SELECT view as view_rating, ROUND(AVG(price),2) as average_price
   FROM home_sales
   GROUP BY view_rating
   ORDER BY view_rating DESC)

SELECT CAST(view_rating AS INTEGER) as view_rating, CAST(average_price AS INTEGER) AS avg_price
FROM pricey_views
WHERE average_price >= 350000
ORDER BY view_rating DESC
"""
).show(60)

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|        100|  1026669|
|         99|  1061201|
|         98|  1053739|
|         97|  1129040|
|         96|  1017815|
|         95|  1054325|
|         94|  1033536|
|         93|  1026006|
|         92|   970402|
|         91|  1137372|
|         90|  1062654|
|         89|  1107839|
|         88|  1031719|
|         87|  1072285|
|         86|  1070444|
|         85|  1056336|
|         84|  1117233|
|         83|  1033965|
|         82|  1063498|
|         81|  1053472|
|         80|   991767|
|         79|  1009565|
|         78|  1080649|
|         77|  1076205|
|         76|  1058802|
|         75|  1114042|
|         74|   745077|
|         73|   752861|
|         72|   780914|
|         71|   775651|
|         70|   695865|
|         69|   750537|
|         68|   716785|
|         67|   737970|
|         66|   712475|
|         65|   736679|
|         64|   767036|
|         63|   711614|
|         62|   

In [51]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [52]:
# 11. Read the parquet formatted data.
hs_df_p=spark.read.parquet('home_sales_partitioned')

In [53]:
# 12. Create a temporary table for the parquet data.
hs_df_p.createOrReplaceTempView('home_sales_p')

In [54]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
WITH pricey_views AS
  (SELECT view as view_rating, ROUND(AVG(price),2) as average_price
   FROM home_sales_p
   GROUP BY view_rating
   ORDER BY view_rating DESC)

SELECT CAST(view_rating AS INTEGER) as view_rating, CAST(average_price AS INTEGER) AS avg_price
FROM pricey_views
WHERE average_price >= 350000
ORDER BY view_rating DESC
"""
).show(60)

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+---------+
|view_rating|avg_price|
+-----------+---------+
|        100|  1026669|
|         99|  1061201|
|         98|  1053739|
|         97|  1129040|
|         96|  1017815|
|         95|  1054325|
|         94|  1033536|
|         93|  1026006|
|         92|   970402|
|         91|  1137372|
|         90|  1062654|
|         89|  1107839|
|         88|  1031719|
|         87|  1072285|
|         86|  1070444|
|         85|  1056336|
|         84|  1117233|
|         83|  1033965|
|         82|  1063498|
|         81|  1053472|
|         80|   991767|
|         79|  1009565|
|         78|  1080649|
|         77|  1076205|
|         76|  1058802|
|         75|  1114042|
|         74|   745077|
|         73|   752861|
|         72|   780914|
|         71|   775651|
|         70|   695865|
|         69|   750537|
|         68|   716785|
|         67|   737970|
|         66|   712475|
|         65|   736679|
|         64|   767036|
|         63|   711614|
|         62|   

In [56]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [58]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False